In [ ]:
import pandas as pd
import os
import boto3
import io
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
bucket_name = "lneg-loka"
key_name = "patient_data_raw/patient_data_raw.csv"
processed_file_name = "patient_data_processed.csv"

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket_name, Key=key_name)
csv_string = obj['Body'].read().decode('utf-8')
df = pd.read_csv(io.StringIO(csv_string))

In [ ]:
df

In [ ]:
df['chronic_obstructive_pulmonary_disease'].unique()

In [ ]:
df['chronic_obstructive_pulmonary_disease'].hist()

In [ ]:
df.isna().sum()

By plotting class-conditional distribution of 'exercise_frequency', it can be seen that it should have little predictive power for prediction chronic obstructive pulmonary disease - the distribution is virtually equal for all classes.

By plotting class-conditional distribution of 'education_level', it can be seen that it should have little predictive power for prediction chronic obstructive pulmonary disease - the distribution is virtually equal for all classes.

Since the distributions of educational_level and exercise_frequency are virtually class-independent, these features should have little predictive power for predicting chronc obstructive pulmonary disease. Because of this, I decided to discard these features instead of other more complex/wasteful solutions to deal with missing/NaN values.

In [ ]:
covariate_name = "smoker"
target="B"
bins_temp = df[df["chronic_obstructive_pulmonary_disease"]==target][covariate_name].unique()
bins = np.array([i for i in bins_temp if str(i).lower() != 'nan'])
bins.sort()
df[df["chronic_obstructive_pulmonary_disease"]==target][covariate_name].value_counts().loc[bins].plot.bar() #use for categorical variables
#df[df["chronic_obstructive_pulmonary_disease"]==target][covariate_name].hist(bins=20) #use for continuous real-valued variables

All variables have equal class-conditional distributions. Should not be possible to get meaningful predictions for chronic obstructive pulmonary disease from these features.
There's almost perfect collinearity between certain features, e.g. BMI and alanine_aminotransferase. Is the data of one of these features corrupted? In any case at least one of these should be dropped

In [ ]:
df_new = df.copy()

Should make preprocessing a part of model pipeline to avoid having to preprocess test data at inference time

In [ ]:
df_new = df_new.drop('exercise_frequency',axis=1)
df_new = df_new.drop('education_level',axis=1)
df_new = df_new.drop('patient_id',axis=1)
df_new = df_new.drop("alanine_aminotransferase",axis=1)

In [ ]:
df_new

In [ ]:
label_encoder = LabelEncoder()
df_new['diet_quality'] = label_encoder.fit_transform(df['diet_quality'])
df_new['income_bracket'] = label_encoder.fit_transform(df['income_bracket'])

In [ ]:
multinomial_categories = ['diagnosis_code']
multinomial_df = pd.get_dummies(df_new[multinomial_categories],drop_first=False)
binomial_categories =['sex','smoker']
binomial_df = pd.get_dummies(df_new[binomial_categories],drop_first=True)
df_new = df_new.drop(multinomial_categories,axis=1)
df_new = df_new.drop(binomial_categories,axis=1)
df_new = pd.concat([df_new,binomial_df],axis=1)
df_new = pd.concat([df_new,multinomial_df],axis=1)
df_new

In [ ]:
buffer = io.StringIO()
df_new.to_csv(buffer)
s3.put_object(Bucket = bucket_name, Key = processed_file_name, Body = buffer.getvalue())